In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
from datasets import load_dataset
import soundfile as sf
import torch
device = torch.device('cuda')
print(device)

# load model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h").to(device)

cuda


/home/mayank/.conda/envs/dss/lib/python3.7/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:419: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
print(type(ds))

<class 'datasets.arrow_dataset.Dataset'>


In [4]:
ds

Dataset({
    features: ['file', 'text', 'speaker_id', 'chapter_id', 'id', 'speech'],
    num_rows: 73
})

In [5]:
type(ds['text'])

list

In [6]:
input_values.shape

torch.Size([2, 104560])

In [7]:
type(input_values)

torch.Tensor

In [8]:
logits.shape

torch.Size([2, 326, 32])

In [9]:
vocab = {"<pad>": 0, "<s>": 1, "</s>": 2, "<unk>": 3, "|": 4, "E": 5, "T": 6, "A": 7, "O": 8, "N": 9, "I": 10, "H": 11, "S": 12, "R": 13, "D": 14, "L": 15, "U": 16, "M": 17, "W": 18, "C": 19, "F": 20, "G": 21, "Y": 22, "P": 23, "B": 24, "V": 25, "K": 26, "'": 27, "X": 28, "J": 29, "Q": 30, "Z": 31}

In [10]:
len(vocab)

32

In [3]:
import re
from unidecode import unidecode
import inflect
import re

# Regular expression matching whitespace:
_whitespace_re = re.compile(r'\s+')

# List of (regular expression, replacement) pairs for abbreviations:
_abbreviations = [(re.compile('\\b%s\\.' % x[0], re.IGNORECASE), x[1]) for x in [
    ('mrs', 'misess'),
    ('mr', 'mister'),
    ('dr', 'doctor'),
    ('st', 'saint'),
    ('co', 'company'),
    ('jr', 'junior'),
    ('maj', 'major'),
    ('gen', 'general'),
    ('drs', 'doctors'),
    ('rev', 'reverend'),
    ('lt', 'lieutenant'),
    ('hon', 'honorable'),
    ('sgt', 'sergeant'),
    ('capt', 'captain'),
    ('esq', 'esquire'),
    ('ltd', 'limited'),
    ('col', 'colonel'),
    ('ft', 'fort'),
]]
def expand_abbreviations(text):
    for regex, replacement in _abbreviations:
        text = re.sub(regex, replacement, text)
    return text

def expand_numbers(text):
    return normalize_numbers(text)

def lowercase(text):
    return text.lower()

def collapse_whitespace(text):
    return re.sub(_whitespace_re, ' ', text)

def convert_to_ascii(text):
    return unidecode(text)

def remove_punctuation(text, table):
    text = text.translate(table)
    text = re.sub(r'&', " and ", text)
    text = re.sub(r'\+', " plus ", text)
    return text

def basic_cleaners(text):
    '''Basic pipeline that lowercases and collapses whitespace without transliteration.'''
    text = lowercase(text)
    text = collapse_whitespace(text)
    return text

def transliteration_cleaners(text):
    '''Pipeline for non-English text that transliterates to ASCII.'''
    text = convert_to_ascii(text)
    text = lowercase(text)
    text = collapse_whitespace(text)
    return text

def english_cleaners(text, table=None):
    '''Pipeline for English text, including number and abbreviation expansion.'''
    text = convert_to_ascii(text)
    text = lowercase(text)
    text = expand_numbers(text)
    text = expand_abbreviations(text)
    if table is not None:
        text = remove_punctuation(text, table)
    text = collapse_whitespace(text)
    return text
   
_inflect = inflect.engine()
_comma_number_re = re.compile(r'([0-9][0-9\,]+[0-9])')
_decimal_number_re = re.compile(r'([0-9]+\.[0-9]+)')
_pounds_re = re.compile(r'£([0-9\,]*[0-9]+)')
_dollars_re = re.compile(r'\$([0-9\.\,]*[0-9]+)')
_ordinal_re = re.compile(r'[0-9]+(st|nd|rd|th)')
_number_re = re.compile(r'[0-9]+')
_time_re = re.compile(r'([0-9]{1,2}):([0-9]{2})')


def _remove_commas(m):
    return m.group(1).replace(',', '')


def _expand_decimal_point(m):
    return m.group(1).replace('.', ' point ')


def _expand_dollars(m):
    match = m.group(1)
    parts = match.split('.')
    if len(parts) > 2:
        return match + ' dollars'  # Unexpected format
    dollars = int(parts[0]) if parts[0] else 0
    cents = int(parts[1]) if len(parts) > 1 and parts[1] else 0
    if dollars and cents:
        dollar_unit = 'dollar' if dollars == 1 else 'dollars'
        cent_unit = 'cent' if cents == 1 else 'cents'
        return '%s %s, %s %s' % (dollars, dollar_unit, cents, cent_unit)
    elif dollars:
        dollar_unit = 'dollar' if dollars == 1 else 'dollars'
        return '%s %s' % (dollars, dollar_unit)
    elif cents:
        cent_unit = 'cent' if cents == 1 else 'cents'
        return '%s %s' % (cents, cent_unit)
    else:
        return 'zero dollars'
    
def _expand_ordinal(m):
    return _inflect.number_to_words(m.group(0))


def _expand_number(m):
    if int(m.group(0)[0]) == 0:
        return _inflect.number_to_words(m.group(0), andword='', group=1)
    num = int(m.group(0))
    if num > 1000 and num < 3000:
        if num == 2000:
            return 'two thousand'
        elif num > 2000 and num < 2010:
            return 'two thousand ' + _inflect.number_to_words(num % 100)
        elif num % 100 == 0:
            return _inflect.number_to_words(num // 100) + ' hundred'
        else:
            return _inflect.number_to_words(num, andword='', zero='oh', group=2).replace(', ', ' ')
    # Add check for number phones and other large numbers
    elif num > 1000000000 and num % 10000 != 0:
        return _inflect.number_to_words(num, andword='', group=1)
    else:
        return _inflect.number_to_words(num, andword='')

def _expand_time(m):
    mins = int(m.group(2))
    if mins == 0:
        return _inflect.number_to_words(m.group(1))
    return " ".join([_inflect.number_to_words(m.group(1)), _inflect.number_to_words(m.group(2))])

def normalize_numbers(text):
    text = re.sub(_comma_number_re, _remove_commas, text)
    text = re.sub(_pounds_re, r'\1 pounds', text)
    text = re.sub(_dollars_re, _expand_dollars, text)
    text = re.sub(_decimal_number_re, _expand_decimal_point, text)
    text = re.sub(_ordinal_re, _expand_ordinal, text)
    text = re.sub(_number_re, _expand_number, text)
    text = re.sub(_time_re, _expand_time, text)
    return text

In [4]:
import string
import toml
import re

#labels: String containing all the possible characters to map to
#blank_index: blank index for ctc loss / decoder

jasper_model_definition = toml.load(base_dir+'../models/pretrained_checkpoints/'+'quartznet/quartznet15x5.toml')
labels = jasper_model_definition['labels']['labels']
# labels = list(map(lambda x: x.lower(), vocab.keys()))
# labels = list(vocab.keys())

print(labels)
blank_index=len(labels)
labels_map = dict([(labels[i], i) for i in range(len(labels))])
blank_index = blank_index
print(labels_map)
print(blank_index)

# def parse_transcript(transcript):
#     chars = [labels_map.get(x, blank_index) for x in list(transcript)]
#     transcript = list(filter(lambda x: x != blank_index, chars))
#     return transcript

# labels = [" ", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "'"]
# print(labels) 
def _clean_text(text, cleaner_names, *args):
#     print(text)
    text = english_cleaners(text, *args)
#     print(text)
    return text


def normalize_string(s):
    """
    Normalizes string. For example:
    'call me at 8:00 pm!' -> 'call me at eight zero pm'

    Args:
        s: string to normalize
        labels: labels used during model training.

    Returns:
            Normalized string
    """

    def good_token(token, labels):
        s = set(labels)
        for t in token:
            if not t in s:
                return False
        return True

    try:
        text = _clean_text(s, ["english_cleaners"], table).strip()
        return ''.join([t for t in text if good_token(t, labels=labels)])
    except:
        print("WARNING: Normalizing {} failed".format(s))
        return None
    
# Punctuation to remove
punctuation = string.punctuation
punctuation = punctuation.replace("+", "")
punctuation = punctuation.replace("&", "")
### We might also want to consider:
### @ -> at
### # -> number, pound, hashtag
### ~ -> tilde
### _ -> underscore
### % -> percent
# If a punctuation symbol is inside our vocab, we do not remove from text
for l in labels:
    punctuation = punctuation.replace(l, "")
# Turn all punctuation to whitespace
table = str.maketrans(punctuation, " " * len(punctuation))


transcript_text = 'call me 8:00 pm!'.upper()
print('original: ', transcript_text)
transcript_text = normalize_string(transcript_text)
print('normalized: ', transcript_text)

# print(parse_transcript(transcript_text)) # convert to vocab indices

[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"]
{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, "'": 27}
28
original:  CALL ME 8:00 PM!
normalized:  call me eight zero zero pm


In [1]:
base_dir = '../../../begin_again/Error-Driven-ASR-Personalization/data/'

In [16]:
import json
import librosa
from tqdm import tqdm

dirs = ['kannada_male_english', 'malayalam_male_english', 'rajasthani_male_english', 'hindi_male_english', 
        'tamil_male_english', 'gujarati_female_english', 'manipuri_female_english', 'assamese_female_english']
base_dir = '../../../begin_again/Error-Driven-ASR-Personalization/data/'
accent = 'tamil_male_english'

def load_audio(file):
    waveform, sr = librosa.load(file, sr=16000)
    return waveform

test_dict = {}
test_file_path = base_dir + accent + '/manifests/test.json'
test_file_list = [json.loads(line.strip()) for line in open(test_file_path)]
test_dict['text'] = [item['text'] for item in test_file_list]
test_dict['file'] = [item['audio_filepath'] for item in test_file_list]
test_dict['duration'] = [item['duration'] for item in test_file_list]
test_dict['speech'] = [load_audio(file) for file in tqdm(test_dict['file'])]
test_dict['n_text'] = [normalize_string(item) for item in test_dict['text']]

print(len(test_dict))

100%|██████████| 2263/2263 [08:56<00:00,  4.22it/s]


5


In [63]:
# import torch
# from torch.utils.data import Dataset, DataLoader

# class indicDataset(Dataset):
#     def __init__(self, _dict):
# #         self.data = {
# #             'text': _dict['text'],
# #             'n_text': _dict['n_text'],
# #             'speech': list(torch.tensor(_dict['speech']))
# #         }
#             self.text= _dict['text']
#             self.n_text= _dict['n_text']
#             self.speech= _dict['speech']
            
#     def __len__(self):
#         return len(self.speech)
# #         return len(self.data['speech'])  

#     def __getitem__(self, idx):
# #         return {
# #                 'speech': self.data['speech'][idx], 
# #                 'n_text': self.data['n_text'][idx],
# #                 'text': self.data['text'][idx]
# #                }
#         return {
#                 'speech': self.speech[idx], 
#                 'n_text': self.n_text[idx],
#                 'text': self.text[idx]
#                }
    
# test_dataset = indicDataset(test_dict)

In [62]:
# next(test_dataset)

In [61]:
# from torch.utils.data import DataLoader

# test_loader = DataLoader(test_dataset, batch_size=1, collate_fn=lambda x: x )
# for batch in tqdm(test_loader):
# #     print(len(batch), type(batch), type(batch[0]), batch[0].keys())
# #     break
#     input_values = tokenizer(batch[0]["speech"], return_tensors="pt", padding="longest").input_values.to(device)  # Batch size 1
#     logits = model(input_values).logits
#     predicted_ids = torch.argmax(logits, dim=-1)
#     transcription = tokenizer.batch_decode(predicted_ids)
#     batch[0]['transcription'] = transcription

In [60]:
# for batch in test_loader:
#     print(batch[0].keys())
#     break

In [17]:
transcription_list = []
for speech in tqdm(test_dict['speech']):
    input_values = tokenizer(speech, return_tensors='pt', padding='longest').input_values.to(device)
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)
    transcription_list.append(transcription[0].lower())
test_dict['transcription'] = transcription_list

100%|██████████| 2263/2263 [02:21<00:00, 15.96it/s]


In [18]:
print(test_dict['transcription'][0], test_dict['n_text'][0])

samon was a viva living on the edge of the city soman was a weaver living on the edge of the city


In [19]:
from jiwer import wer

wer(test_dict['transcription'], test_dict['n_text'])

0.09129150630985411

In [1]:
!export CUDA_VISIBLE_DEVICES=1

In [8]:
!nvidia-smi

Tue Aug 24 06:45:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:3B:00.0 Off |                  N/A |
| 35%   47C    P8    10W / 250W |  11167MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  On   | 00000000:5E:00.0 Off |                  N/A |
| 59%   

In [59]:
###for manipuri_female
for i in range(0, len(test_dict['text']), 10):
    print('ref:', test_dict['n_text'][i])
    print('hyp:', test_dict['transcription'][i])
    print()

ref: among them
hyp: among them

ref: by exercising
hyp: by exercising

ref: the ship should be in within a week
hyp: tha see soot be in weeding a wee

ref: the husband sow the paddy
hyp: the husband so the petty

ref: to the students who could not apply
hyp: to thes to dense who could not apply

ref: hands enter the house
hyp: haines enter the house

ref: in harsh words
hyp: in hears worte

ref: from that moment his friendship
hyp: from that momen he's prency

ref: she broke into hearty laughter
hyp: sea broke into harty laughter

ref: widely so as to drop more delicious fruits
hyp: whitely so as to draw more deliceous pruits

ref: of these substances may worsen your symptoms
hyp: of these substancees may worsen your symptoms

ref: were fruitless
hyp: were bruitless

ref: j paite president of kuki peoples front
hyp: j pite preciden o cooki people's pran

ref: in god's bountiful providence
hyp: in gods bauntifool providence

ref: the foolish tiger
hyp: the fullest tiger

ref: heron and